In [7]:
from flo_ai import Flo
from flo_ai import FloSession
from langchain_openai import ChatOpenAI, OpenAIEmbeddings

from dotenv import load_dotenv
load_dotenv()

True

In [8]:
from langchain_community.tools.tavily_search.tool import TavilySearchResults

llm = ChatOpenAI(temperature=0, model_name='gpt-4o-mini')
session = FloSession(llm)

session.register_tool(
    name="InternetSearchTool", 
    tool=TavilySearchResults()
)

In [9]:
simple_weather_checking_agent = """
apiVersion: flo/alpha-v1
kind: FloAgent
name: weather-assistant
agent:
    name: Weather Assistant
    job: >
      Given the city name you are capable of answering the latest whether this time of the year by searching the internet
    tools:
      - name: InternetSearchTool
"""

In [6]:
flo = Flo.build(session, simple_weather_checking_agent)

print(flo.invoke("Whats the whether in california"))



> Entering new AgentExecutor chain...

Invoking: `tavily_search_results_json` with `{'query': 'California weather October 2023'}`


[{'url': 'https://www.weatherapi.com/', 'content': "{'location': {'name': 'California City', 'region': 'California', 'country': 'United States of America', 'lat': 35.13, 'lon': -117.99, 'tz_id': 'America/Los_Angeles', 'localtime_epoch': 1725354983, 'localtime': '2024-09-03 02:16'}, 'current': {'last_updated_epoch': 1725354900, 'last_updated': '2024-09-03 02:15', 'temp_c': 26.1, 'temp_f': 79.0, 'is_day': 0, 'condition': {'text': 'Clear', 'icon': '//cdn.weatherapi.com/weather/64x64/night/113.png', 'code': 1000}, 'wind_mph': 4.3, 'wind_kph': 6.8, 'wind_degree': 160, 'wind_dir': 'SSE', 'pressure_mb': 1016.0, 'pressure_in': 29.99, 'precip_mm': 0.0, 'precip_in': 0.0, 'humidity': 26, 'cloud': 0, 'feelslike_c': 25.5, 'feelslike_f': 77.9, 'windchill_c': 24.1, 'windchill_f': 75.4, 'heatindex_c': 24.3, 'heatindex_f': 75.7, 'dewpoint_c': 5.8, 'dewpoint_f': 42.4, 'vi

In [3]:
simple_ll_agent = """
apiVersion: flo/alpha-v1
kind: FloAgent
name: llm-assistant
agent:
    name: Ask llm anything
    kind: llm
    job: >
      You are a high school maths teacher. Answer any questions the students ask 
"""

In [4]:
flo = Flo.build(session, simple_ll_agent)

print(flo.invoke("What is pythagorus theorum"))

Pythagoras' Theorem is a fundamental principle in geometry that relates to right-angled triangles. It states that in a right triangle, the square of the length of the hypotenuse (the side opposite the right angle) is equal to the sum of the squares of the lengths of the other two sides. 

The theorem can be expressed with the formula:

\[ c^2 = a^2 + b^2 \]

where:
- \( c \) is the length of the hypotenuse,
- \( a \) and \( b \) are the lengths of the other two sides.

This theorem is very useful for calculating distances and solving problems involving right triangles. If you have any specific examples or problems you'd like to discuss, feel free to ask!
